In [3]:
import os
import threading
import time
import subprocess
import sys
from google.colab import userdata

# 1. Install Dependencies First
print("Installing Dependencies (pyngrok & Ollama)...")

# Install pyngrok using the current python executable
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyngrok"], check=True)

# Install Ollama
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=True)

# 2. Import Libraries
# Now that it is installed, we can safely import it
from pyngrok import ngrok, conf

# 3. Load Security Token
try:
    ngrok_token = userdata.get('NGROK_TOKEN')
    if not ngrok_token:
        raise ValueError("Token is empty")
    print("Successfully loaded ngrok token from secrets.")
except Exception as e:
    print("ERROR: Could not find 'NGROK_TOKEN' in Secrets!")
    print("Click the Key icon on the left sidebar.")
    print("Add a secret named 'NGROK_TOKEN' with your value.")
    print("Toggle 'Notebook access' to ON.")
    raise e

# Authenticate ngrok
conf.get_default().auth_token = ngrok_token

# 4. Start Ollama Server
print("Starting Ollama Server...")
def run_ollama():
    # Set environment variables to allow access
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.run(["ollama", "serve"])

# Run in background thread
thread = threading.Thread(target=run_ollama)
thread.start()

# Wait for server to be responsive
print("Waiting for Ollama to spin up (10s)...")
time.sleep(10)

# 5. Download Model
print("Downloading Llama 3:latest... (This takes ~1-2 mins)")
subprocess.run(["ollama", "pull", "llama3:latest"], check=True)

# 6. Create Public Tunnel
print("Creating Public Tunnel...")

# Close old tunnels
ngrok.kill()

# Create new tunnel
try:
    # Open port 11434
    public_url = ngrok.connect(11434).public_url

    print("\n" + "="*60)
    print("SERVER READY! Update your backend/.env file:")
    print("-" * 60)
    print(f"OLLAMA_BASE_URL={public_url}")
    print(f"OLLAMA_MODEL=llama3:latest")
    print("-" * 60)
    print("Keep this tab OPEN. If you close it, the URL dies.")
    print("="*60)

    # Keep cell alive
    while True:
        time.sleep(1)

except Exception as e:
    print(f"Error creating tunnel: {e}")

Installing Dependencies (pyngrok & Ollama)...
Successfully loaded ngrok token from secrets.
Starting Ollama Server...
Waiting for Ollama to spin up (10s)...
Creating Public Tunnel...

SERVER READY! Update your backend/.env file:
------------------------------------------------------------
OLLAMA_BASE_URL=https://reducibly-beauish-dana.ngrok-free.dev
OLLAMA_MODEL=llama3:latest
------------------------------------------------------------
Keep this tab OPEN. If you close it, the URL dies.


KeyboardInterrupt: 